In [1]:
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline
import mpld3
#mpld3.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

- On convertit le score en distance avec $d=\frac{1}{score}10^4$.
- On considère alors que chaque fonction de similarité défini une distance selon une dimension par rapport à la sonde.
- On va donc apprendre une métrique dans un espace à 8 dimensions, de façon a rapprocher de la sonde les labels identiques à celui de la sonde, et à éloigner les labels différents.



## preprocesings

La première étape est de mettre les données dans un format plus pratique: 
- pour chaque sonde, identifier les Ids présents dans les 8 similarités
- ranger les données dans un tableau | Ids | distance 1 | ... | distance 8 |
- éventuellement, pour chaque sonde, ne garder que les n plus proches labels en distance euclidienne (on a vu dans le notebook premiers tests, que les 7 premières lignes des scores contiennent majoritairement le bon label)

In [2]:
train = io.loadmat('/home/max/projects/challengeMDI343/data/data_train.mat')

In [7]:
probes = { train['probeId'][i][0]: train['probeLabel'][i][0] for i in range(len(train['probeLabel']))}

print probes[7490]

gallery = {train['galleryId'][i][0]: train['galleryLabel'][i][0] for i in range(len(train['galleryLabel']))}

print gallery[3]

13721
45407


In [54]:
resultsId = train['resultsId']
resultsScore = train['resultsScore']

probe = 1001

uniques, counts = np.unique(resultsId[probe,:,:], return_counts=True)

print uniques
print counts

valids = uniques[counts == 6]

print 'label:', gallery[valids[0]], train['probeLabel'][probe][0]
print valids.shape


 [   118    273    399    505   1549   1847   1850   2337   2564   2568
   2888   3521   3529   3871   4132   4237   4242   4499   5077   5083
   5218   5247   5346   5629   5922   6010   6037   6103   6200   6224
   6387   6452   6468   6580   7232   7346   7478   7533   7550   7867
   8198   8366   8489   8892   8931   8944   9286   9351   9486   9491
  10019  10533  10550  10668  10707  11136  11209  11461  11560  11691
  11698  11876  11957  11975  12052  12097  12248  12348  12445  12578
  12725  12758  13184  13689  13705  13874  14332  14748  15113  15719
  15960  16043  16181  16229  16407  16471  16878  17145  17408  17739
  18164  18204  18605  18638  18684  19631  19960  20107  20171  20516
  20602  20706  21115  21327  21345  21407  21512  21617  21796  21996
  22102  22405  22647  22862  23118  23777  23780  24064  24267  24301
  24509  24554  24612  24763  24867  25556  25680  26047  26345  26906
  27377  27902  27914  28024  28628  29024  29180  29560  30262  30282
  305

In [51]:
probe = 1001

test = np.asarray([[gallery[resultsId[probe,i,j]] == train['probeLabel'][probe][0] for j in range(7)] for i in range(8)])

print test

[[ True False False False False False False]
 [ True False False False False False False]
 [False  True False False False False False]
 [False False False  True False False False]
 [False False False False False False False]
 [False False False False False False False]
 [False False False False False  True False]
 [ True False False False False False False]]
